In [ ]:
from watermark import watermark
from pathlib import Path
import pandas as pd
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 2000)
pd.options.display.float_format = '{:0,.2f}'.format #avoid scientific notation, and thousands separator 
import numpy as np

import hvplot.pandas
import seaborn as sns
hvplot.extension('bokeh')
%matplotlib inline

import sklearn as sk
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

## Data cleanup and high-level analysis

In [ ]:
#System Requirements

print(watermark())
print(watermark(iversions=True, globals_=globals()))

In [ ]:
# Read data from the Kaggle database file into a Pandas DataFrame
df = pd.read_csv(
    Path("../../LG_Resources/Resources/lending-club/accepted_2007_to_2018Q4.csv/accepted_2007_to_2018Q4.csv"),  
    infer_datetime_format=True,
    parse_dates = True,
    low_memory=False
)

# Review the DataFrame
df.head()

In [ ]:
#Check number of NAN values in each column

nan_values = pd.DataFrame(df.isna().sum(),columns = ["NAN Count"]).reset_index()
# nan_values.sort_values(nan_values.columns[1], ascending=False) #commented out for memory use

In [ ]:
#Check average number of NAN values and how many columns have NAN values over 500k
display(nan_values[(nan_values['NAN Count'] > 500000)].shape)
nan_values['NAN Count'].mean()

In [ ]:
#Drop columns with over 500k NAN values + ID column (not relevant) and create new dataframe

drop_columns = nan_values[(nan_values['NAN Count'] > 500000)]['index'].tolist() 
drop_columns.extend(['id','url','title','zip_code']) #deemed unnecessary after review 
df = df.drop(drop_columns, axis=1)
df.isna().sum().mean()
# drop_columns

In [ ]:
#Reorder columns in alphabetical order and print list of columns for further data analysis
ordercolumns = df.columns.tolist()
ordercolumns.sort()
df = df[ordercolumns]
df.columns

## Adjust df to load into model

In [ ]:
#create list with all loan status values
status = df['loan_status'].dropna().unique().tolist()
status

In [ ]:
# create list with values considered default (i.e. Y value)
defaultstatus = status[2:-2]
defaultstatus.append(status[-1])
defaultstatus

In [ ]:
# check for items not considered default 
goodstatus = [i for i in status if i not in defaultstatus]
goodstatus

In [ ]:
df['Default'] = np.where(df['loan_status'].isin(defaultstatus), 1, 0)

In [ ]:
#checking all Y value loans have been tagged correctly
print(df[df['Default']==1]['loan_status'].isin(defaultstatus).value_counts())

In [ ]:
# visual check eveyrthing was tagged correctly
check = pd.DataFrame(df['loan_status'].value_counts())
check['defaults'] = df[df['Default'] == 1]['loan_status'].value_counts()
check

In [ ]:
# df.drop(['loan_status'],axis=1,inplace=True)
# df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'])
# df['issue_d'] = pd.to_datetime(df['issue_d'])
# df['last_credit_pull_d'] = pd.to_datetime(df['last_credit_pull_d'])
# df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'])
# df.drop(['emp_title'],axis=1,inplace=True)
df.drop(columns= ['emp_title','earliest_cr_line','issue_d','last_credit_pull_d','last_pymnt_d','Default','loan_status'],inplace=True)

In [ ]:
changetypes = pd.DataFrame(df.dtypes,columns = ['type']).reset_index()
changetypes = changetypes[changetypes['type']=='object']
changetypes

In [ ]:
newdict = {}
for colname in changetypes["index"]:
    counter = 0.0
    col_dict = {}
    for elem in df[colname].unique():
        col_dict[elem] = counter
        counter = counter+1
    newdict[colname] = col_dict

In [ ]:
newdict

In [ ]:
#slight manual adjustments to dictionary above to remove nan values 
cleaner = {'addr_state': {'PA': 0.0,  'SD': 1.0,  'IL': 2.0,  'NJ': 3.0,
  'GA': 4.0,  'MN': 5.0,  'SC': 6.0,  'RI': 7.0,  'TX': 8.0,  'NC': 9.0,  'CA': 10.0,  'VA': 11.0,
  'AZ': 12.0,  'NY': 13.0,  'IN': 14.0,  'MD': 15.0,  'KS': 16.0,  'NM': 17.0,  'AL': 18.0,  'WA': 19.0,
  'MO': 20.0,  'OH': 21.0,  'LA': 22.0,  'FL': 23.0,  'CO': 24.0,  'MI': 25.0,  'TN': 26.0,
  'DC': 27.0,  'MA': 28.0,  'WI': 29.0,  'HI': 30.0,  'VT': 31.0,  'DE': 32.0,  'NH': 33.0,  'NE': 34.0,
  'CT': 35.0,  'OR': 36.0,  'AR': 37.0,  'MT': 38.0,  'NV': 39.0,  'WV': 40.0,  'WY': 41.0,  'OK': 42.0,
  'KY': 43.0,  'MS': 44.0,  'ME': 45.0,  'UT': 46.0,  'ND': 47.0,  'AK': 48.0,  'ID': 50.0,  'IA': 51.0},
 'application_type': {'Individual': 0.0, 'Joint App': 1.0},
 'debt_settlement_flag': {'N': 0.0, 'Y': 1.0},
 'disbursement_method': {'Cash': 1.0, 'DirectPay': 2.0},
 'emp_length': {'10+ years': 10.0,  '3 years': 3.0,  '4 years': 4.0,
  '6 years': 6.0,  '1 year': 1.0,  '7 years': 7.0,  '8 years': 8.0,
  '5 years': 5.0,  '2 years': 2.0,  '9 years': 9.0,  '< 1 year': 1.0},
 'grade': {'C': 0.0,  'B': 1.0,  'F': 2.0,  'A': 3.0,  'E': 4.0,  'D': 5.0,  'G': 6.0},
 'hardship_flag': {'N': 0.0, 'Y': 1.0},
 'home_ownership': {'MORTGAGE': 0.0,  'RENT': 1.0,
      'OWN': 2.0,  'ANY': 3.0,'NONE': 4.0,  'OTHER': 5.0},
 'initial_list_status': {'w': 0.0, 'f': 1.0},
 'purpose': {'debt_consolidation': 0.0,  'small_business': 1.0,
      'home_improvement': 2.0,  'major_purchase': 3.0,
      'credit_card': 4.0,  'other': 5.0,
      'house': 6.0,  'vacation': 7.0,
      'car': 8.0,  'medical': 9.0,
      'moving': 10.0,  'renewable_energy': 11.0,
      'wedding': 12.0,  'educational': 13.0,},
 'pymnt_plan': {'n': 0.0, 'y': 1.0},
 'sub_grade': {'C4': 0.0,  'C1': 1.0,
      'B4': 2.0,  'C5': 3.0,
      'F1': 4.0,  'C3': 5.0,
      'B2': 6.0,  'B1': 7.0,
      'A2': 8.0,  'B5': 9.0,
      'C2': 10.0,  'E2': 11.0,
      'A4': 12.0,  'E3': 13.0,
      'A1': 14.0,  'D4': 15.0,
      'F3': 16.0,  'D1': 17.0,
      'B3': 18.0,  'E4': 19.0,
      'D3': 20.0,  'D2': 21.0,
      'D5': 22.0,  'A5': 23.0,
      'F2': 24.0,  'E1': 25.0,
      'F5': 26.0,  'E5': 27.0,
      'A3': 28.0,  'G2': 29.0,
      'G1': 30.0,  'G3': 31.0,
      'G4': 32.0,  'F4': 33.0,
      'G5': 34.0},
 'term': {' 36 months': 0.0, ' 60 months': 1.0},
 'verification_status': {'Not Verified': 0.0,
  'Source Verified': 1.0,
  'Verified': 2.0}}

In [ ]:
modeldf = df.replace(cleaner)
modeldf = modeldf.fillna(0.00)

In [ ]:
modeldf.dtypes

## Run K Means (unscaled)

In [ ]:
model_k4 = KMeans(n_clusters=4)

In [ ]:
model_k4.fit(modeldf)

In [ ]:
loan_segments_k4 = model_k4.predict(modeldf)
print(loan_segments_k4)

In [ ]:
model_k6 = KMeans(n_clusters=6)

In [ ]:
model_k6.fit(modeldf)

In [ ]:
loan_segments_k6 = model_k6.predict(modeldf)
print(loan_segments_k6)

In [ ]:
kmodel_predictions = modeldf.copy()
kmodel_predictions['Segments k=4'] = loan_segments_k4
kmodel_predictions['Segments k=6'] = loan_segments_k6
kmodel_predictions.head()

In [ ]:
k4plot = kmodel_predictions.hvplot.scatter(
    x="annual_inc", 
    y="fico_range_high", 
    by="Segments k=4",
    title = "Scatter Plot by Segment - k=4"
)

k6plot = kmodel_predictions.hvplot.scatter(
    x="annual_inc", 
    y="fico_range_high", 
    by="Segments k=6",
    title = "Scatter Plot by Segment - k=6"
)


k4plot + k6plot

## Scale data

In [ ]:
#separate categorical (ones we assigned numebers to) from non-categorical columns
categorical = [i for i in changetypes['index']]
categorical

In [ ]:
noncategorical = [i for i in modeldf.columns if i not in categorical]
noncategorical

In [ ]:
#scale data
scaled_data = StandardScaler().fit_transform(modeldf[noncategorical])
scaled_data[0:1]

In [ ]:
#create a new scaled df
modeldf_scaled = pd.DataFrame(scaled_data)
modeldf_scaled.columns = noncategorical
modeldf_scaled.head()

In [ ]:
#add back ncategorical data
modeldf_scaled[categorical] = modeldf[categorical]
modeldf_scaled.head()

In [ ]:
# columnames = noncategorical
# columnames.extend(categorical)
# columnames

In [ ]:
# modeldf_scaled.columns = columnames
modeldf_scaled.columns

## Run K Means (scaled)

In [ ]:
model_k4 = KMeans(n_clusters=4)

In [ ]:
model_k4.fit(modeldf_scaled)

In [ ]:
loan_segments_k4 = model_k4.predict(modeldf_scaled)
print(loan_segments_k4)

In [ ]:
model_k6 = KMeans(n_clusters=6)

In [ ]:
model_k6.fit(modeldf_scaled)

In [ ]:
loan_segments_k6 = model_k6.predict(modeldf_scaled)
print(loan_segments_k4)

In [ ]:
sc_kmodel_predictions = modeldf_scaled.copy()
sc_kmodel_predictions['Segments k=4'] = loan_segments_k4
sc_kmodel_predictions['Segments k=6'] = loan_segments_k6
sc_kmodel_predictions.head()

In [ ]:
sc_k4plot = sc_kmodel_predictions.hvplot.scatter(
    x="int_rate", 
    y="fico_range_high", 
    by="Segments k=4",
    title = "Scatter Plot by Segment - k=4"
)

sc_k6plot = sc_kmodel_predictions.hvplot.scatter(
    x="int_rate", 
    y="fico_range_high", 
    by="Segments k=6",
    title = "Scatter Plot by Segment - k=6"
)


sc_k4plot + sc_k6plot